# Get_qaoa_results

## Setup

In [1]:
import os
os.chdir("..")

In [2]:
from config import code_path, results_path, data_path, seed, mixer, method, verbose

In [3]:
os.chdir(code_path)
from classes import Qaoa as Q
from classes import Problems as P
from functions import qaoa_utilities as qaoa_utils
from functions import maxcut_utilities as mcut_utils
from functions import optimizers as optims

In [4]:
import numpy as np
import random
import pickle
import sys

## Data laoding

In [5]:
# LOAD THE GRAPHS 

graphs = []
for i in range(18):
    graph_path =  "/graph_"+str(i)+".nx"
    with open(data_path + graph_path, 'rb') as f:
        g = pickle.load(f)
    graphs.append(g)

In [6]:
# LOAD BRUTE FORCE MAXCUT RESULTS

with open(results_path + "/brute_force_results.pkl", 'rb') as f:
    bfs = pickle.load(f)

## Getting the results

In [7]:
os.chdir(results_path)

### QAOA vanilla

In [8]:
for p in [1,2,3,4,5,6,7,8]:
    betas = qaoa_utils.generate_parameters(n=p, k=1, seed=seed)
    gammas = qaoa_utils.generate_parameters(n=p, k=2, seed=seed)
    for idx in range(16):
        name = "qaoa_vanilla_graph_" + str(idx) + "_p_" + str(p) + "_seed_" + str(seed)
        G = graphs[idx]
        qaoa = Q.Qaoa(p=p, G=G, betas=betas, gammas=gammas, mixer=mixer, seed=seed, verbose=verbose)
        x, f = optims.simple_optimization(qaoa, method=method, seed=seed, verbose=verbose)
        AR = -f/bfs[idx]
        with open(name, 'wb') as file:
            pickle.dump((x,f,AR), file)

### QAOA extended with one "shadow" node

In [9]:
for p in [1,2,3,4,5,6,7,8]:
    betas = qaoa_utils.generate_parameters(n=p, k=1, seed=seed)
    gammas = qaoa_utils.generate_parameters(n=p, k=2, seed=seed)
    for idx in range(18):
        name = "qaoa_extended_1_graph_" + str(idx) + "_p_" + str(p) + "_seed_" + str(seed)
        G = graphs[idx]
        N = graphs[idx].get_number_of_nodes()
        G_ = graphs[idx].get_graph().copy()
        G_.add_node(N)
        extended_graph_instance = P.Problems(p_type="custom", G=G_)
        qaoa = Q.Qaoa(p=p, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed, verbose=verbose)
        x, f = optims.simple_optimization(qaoa, method=method, seed=seed, verbose=verbose)
        AR = -f/bfs[idx]
        with open(name, 'wb') as file:
            pickle.dump((x,f,AR), file)

### QAOA extended with two "shadow" nodes

In [10]:
for p in [1,2,3,4,5,6,7,8]:
    betas = qaoa_utils.generate_parameters(n=p, k=1, seed=seed)
    gammas = qaoa_utils.generate_parameters(n=p, k=2, seed=seed)
    for idx in range(18):
        name = "qaoa_extended_2_graph_" + str(idx) + "_p_" + str(p) + "_seed_" + str(seed)
        G = graphs[idx]
        N = graphs[idx].get_number_of_nodes()
        G_ = graphs[idx].get_graph().copy()
        G_.add_node(N)
        G_.add_node(N+1)
        extended_graph_instance = P.Problems(p_type="custom", G=G_)
        qaoa = Q.Qaoa(p=p, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed, verbose=verbose)
        x, f = optims.simple_optimization(qaoa, method=method, seed=seed, verbose=verbose)
        AR = -f/bfs[idx]
        with open(name, 'wb') as file:
            pickle.dump((x,f,AR), file)

### QAOA extended with one pendent edge

In [11]:
for p in [1,2,3,4,5,6,7,8]:
    betas = qaoa_utils.generate_parameters(n=p, k=1, seed=seed)
    gammas = qaoa_utils.generate_parameters(n=p, k=2, seed=seed)
    for idx in range(18):
        name = "qaoa_extended_3_graph_" + str(idx) + "_p_" + str(p) + "_seed_" + str(seed)
        G = graphs[idx]
        N = graphs[idx].get_number_of_nodes()
        G_ = graphs[idx].get_graph().copy()
        G_.add_node(N)
        u = list(G_.nodes())[N]
        v = list(G_.nodes())[random.randint(0,N-1)]
        G_.add_edge(u,v)
        extended_graph_instance = P.Problems(p_type="custom", G=G_)
        qaoa = Q.Qaoa(p=p, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed, verbose=verbose)
        x, _ = optims.simple_optimization(qaoa, method=method, seed=seed, verbose=verbose)
        qaoa = Q.Qaoa(p=p, G=graphs[idx], betas=x[:p], gammas=x[p:], mixer=mixer, seed=seed, verbose=verbose)
        _, f = optims.simple_optimization(qaoa, method=method, seed=seed, verbose=verbose)
        AR = -f/bfs[idx]
        with open(name, 'wb') as file:
            pickle.dump((x,f,AR), file)

### QAOA reduced

In [12]:
for p in [1,2,3,4,5,6,7,8]:
    betas = qaoa_utils.generate_parameters(n=p, k=1, seed=seed)
    gammas = qaoa_utils.generate_parameters(n=p, k=2, seed=seed)
    for idx in range(18):
        name = "qaoa_reduced_graph_" + str(idx) + "_p_" + str(p) + "_seed_" + str(seed)
        G = graphs[idx]
        G_ = graphs[idx].get_graph().copy()
        G_prime = P.Problems(p_type="custom", G=G_)
        opt = [0]
        for edge in G_prime.get_edges():
            G_reduced = G_prime.get_graph().copy()
            G_reduced.remove_edges_from([edge])
            reduced_graph_instance = P.Problems(p_type="custom", G=G_reduced)
            qaoa = Q.Qaoa(p=p, G=G, betas=betas, gammas=gammas, mixer=mixer, seed=seed, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed, verbose=verbose)
            AR = -f/bfs[idx]
            opt.append(AR)
        with open(name, 'wb') as file:
            pickle.dump((x,f,max(opt)), file)